The goal of this project is connect to a website called "Buscalibre" which is an only library and pull in the data related with an specific search of books, categories, etc.
Then clean and create a DataFrame to save the data such as the book name, author, price, edition etc, into a csv file.


In [1]:
# Import libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
import csv
import time

In [2]:
# Connect to website and pull in and cleaning the data 
# Create the lists to save the data
id_tags= []
links = []
Titles = []
Authors = []
Num_Opinions = []
Info = []
Units_Left = []
Discount = []
Original_Price = []
Actual_Price = []
# The url after the searching the 'siddartha' book
url = 'https://www.buscalibre.com.co/libros/search?q=siddartha'
response = requests.get(url = url)
soup = BeautifulSoup(response.content,"html.parser")
soup1 = BeautifulSoup(soup.prettify(),"html.parser")

# It is possible that one search throw a few or hundred and thousand of results (books), and the website does not show all the results (if they are many) in one page, so the condition below
# ensure when is necesary to pull the data from multiple pages in the website or just one page.

if (soup1.find(name='a',id='pagnNextLink') != None):
    while(True):
        try:
            
            response = requests.get(url = url)
            soup = BeautifulSoup(response.content,"html.parser")
            soup1 = BeautifulSoup(soup.prettify(),"html.parser")

            next_url = soup1.find(name='a',id='pagnNextLink').get('href')
            url = next_url
            all_Products = soup1.find(name='div', class_='productos pais46')
            all_Tags = all_Products.find_all('div')

            for tag in all_Tags:
                try:
                    
                    id_tags.append(tag['data-id_producto'])
                    links.append(tag.find('a').get('href'))
                    Titles.append(tag.find(name='h3',class_='nombre margin-top-10 text-align-left').get_text().strip())
                    Authors.append(tag.find(name='div',class_='autor').get_text().strip())
                    Num_Opinions.append(tag.find(name='div',class_='box-stars hide-on-hover').get_text().strip())
                    Info.append(tag.find(name='div',class_='autor color-dark-gray metas hide-on-hover').get_text().strip())
                    Units_Left.append(tag.find(name='div',class_='stock hide-on-hover color-green font-size-small margin-top-5').get_text().strip())
                    Discount.append(tag.find(name='div',class_='descuento-v2 color-white position-relative').get_text().strip())
                    Original_Price.append(tag.find(name='p',class_='precio-antes hide-on-hover margin-0 color-dark-gray font-weight-normal').get_text().strip())
                    if tag.find(name='p',class_='precio-ahora hide-on-hover margin-0 font-size-medium') != None:
                        Actual_Price.append(tag.find(name='p',class_='precio-ahora hide-on-hover margin-0 font-size-medium').get_text().strip())
                    else: 
                        Actual_Price.append('Sold Out')
                    
                except KeyError:
                    continue
        except:
            break
else: 
    all_Products = soup1.find(name='div', class_='productos pais46')   
    all_Tags = all_Products.find_all('div')

    for tag in all_Tags:
        try:
            id_tags.append(tag['data-id_producto'])
            links.append(tag.find('a').get('href'))
            Titles.append(tag.find(name='h3',class_='nombre margin-top-10 text-align-left').get_text().strip())
            Authors.append(tag.find(name='div',class_='autor').get_text().strip())
            Num_Opinions.append(tag.find(name='div',class_='box-stars hide-on-hover').get_text().strip())
            Info.append(tag.find(name='div',class_='autor color-dark-gray metas hide-on-hover').get_text().strip())
            Units_Left.append(tag.find(name='div',class_='stock hide-on-hover color-green font-size-small margin-top-5').get_text().strip())
            Discount.append(tag.find(name='div',class_='descuento-v2 color-white position-relative').get_text().strip())
            Original_Price.append(tag.find(name='p',class_='precio-antes hide-on-hover margin-0 color-dark-gray font-weight-normal').get_text().strip())
            if tag.find(name='p',class_='precio-ahora hide-on-hover margin-0 font-size-medium') != None:
                Actual_Price.append(tag.find(name='p',class_='precio-ahora hide-on-hover margin-0 font-size-medium').get_text().strip())
            else: 
                Actual_Price.append('Sold Out')
        except KeyError:
                    continue
                    

In [3]:
# Clean up the data a little bit

Clean_Discount = []
for j in range(0,len(Discount)):
    Discount[j] = Discount[j].split('%')
    Discount[j][-1] = Discount[j][-1].strip()
    Clean_Discount.append(Discount[j][0])

    Info[j] =Info[j].split(',')
    Info[j][-1] = Info[j][-1].strip()

    Original_Price[j] = Original_Price[j].replace('$','')
    Actual_Price[j] = Actual_Price[j].replace('$','') 

    if Num_Opinions[j] == '':
        Num_Opinions[j] = '(0)'
        


In [4]:
# Create the DataFrame with all the data 

Nombre_Columnas = ['id_tags','Title','Author','Original Price (COP)','Actual Price (COP)','Discount %','Units Left','Link','Number of Opinions','Extra Information']
df = pd.DataFrame(list(zip(id_tags,Titles,Authors,Original_Price,Actual_Price,Clean_Discount,Units_Left,links,Num_Opinions,Info)),columns=Nombre_Columnas)
df['Date'] = datetime.date.today()
df

,id_tags,Title,Author,Original Price (COP),Actual Price (COP),Discount %,Units Left,Link,Number of Opinions,Extra Information,Date
0,32485405,Siddartha,Hesen,28.000,26.823,4,Quedan 50 unidades,https://www.buscalibre.com.co/libro-siddartha/...,(0),"[, Nuevo]",2023-11-06
1,53434381,Siddartha,Array,100.766,65.498,35,Quedan 100+ unidades,https://www.buscalibre.com.co/libro-siddartha/...,(0),"[Createspace Independent Publishing Platform, ...",2023-11-06
2,17946638,Siddhartha (Ed. Escolar),Hermann Hesse,46.000,32.200,30,Quedan 79 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(12),"[Debolsillo, 2010, Tapa blanda, Nuevo]",2023-11-06
3,52804400,Siddhartha,Hermann Hesse,54.000,52.329,,Quedan 76 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(15),"[Debolsillo, 2020, 1ª edición, Tapa dura, N...",2023-11-06
4,48128579,Siddhartha. Alma de Niño,Hermann Hesse,105.998,68.899,35,Quedan 100+ unidades,https://www.buscalibre.com.co/libro-siddhartha...,(7),"[Zig-Zag, 2013, Tapa Blanda, Nuevo]",2023-11-06
...,...,...,...,...,...,...,...,...,...,...,...
268,54940141,Human,"Bhattacharyya, Siddhartha",385.210,250.386,35,Quedan 2 unidades,https://www.buscalibre.com.co/libro-human-cent...,(0),"[Springer 2022-11, , Tapa Blanda, Nuevo]",2023-11-06
269,46598250,Statutory Auditors' Objectivity in Corporate A...,Siddhartha Sankar Saha,549.977,357.485,35,Quedan 50 unidades,https://www.buscalibre.com.co/libro-statutory-...,(0),"[Lap Lambert Academic Publishing, Tapa blanda...",2023-11-06
270,46631218,Regulating Cell to Cell Gap,Siddhartha Mitra,608.492,395.520,35,Quedan 50 unidades,https://www.buscalibre.com.co/libro-regulating...,(0),"[Lap Lambert Acad. Publ., Tapa blanda, Nuevo]",2023-11-06
271,46652869,UHF/VHF Radars studies on Atmospheric Stable a...,Siddarth Shankar Das,538.652,350.124,35,Quedan 50 unidades,https://www.buscalibre.com.co/libro-uhf-vhf-ra...,(0),"[Lap Lambert Academic Publishing, Tapa blanda...",2023-11-06


In [5]:
# DataFrame without the Sold Out books

index_toDrop = df[df['Actual Price (COP)']=='Sold Out'].index
df_copy = df.drop(index_toDrop)
df_copy

,id_tags,Title,Author,Original Price (COP),Actual Price (COP),Discount %,Units Left,Link,Number of Opinions,Extra Information,Date
0,32485405,Siddartha,Hesen,28.000,26.823,4,Quedan 50 unidades,https://www.buscalibre.com.co/libro-siddartha/...,(0),"[, Nuevo]",2023-11-06
1,53434381,Siddartha,Array,100.766,65.498,35,Quedan 100+ unidades,https://www.buscalibre.com.co/libro-siddartha/...,(0),"[Createspace Independent Publishing Platform, ...",2023-11-06
2,17946638,Siddhartha (Ed. Escolar),Hermann Hesse,46.000,32.200,30,Quedan 79 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(12),"[Debolsillo, 2010, Tapa blanda, Nuevo]",2023-11-06
3,52804400,Siddhartha,Hermann Hesse,54.000,52.329,,Quedan 76 unidades,https://www.buscalibre.com.co/libro-siddhartha...,(15),"[Debolsillo, 2020, 1ª edición, Tapa dura, N...",2023-11-06
4,48128579,Siddhartha. Alma de Niño,Hermann Hesse,105.998,68.899,35,Quedan 100+ unidades,https://www.buscalibre.com.co/libro-siddhartha...,(7),"[Zig-Zag, 2013, Tapa Blanda, Nuevo]",2023-11-06
...,...,...,...,...,...,...,...,...,...,...,...
268,54940141,Human,"Bhattacharyya, Siddhartha",385.210,250.386,35,Quedan 2 unidades,https://www.buscalibre.com.co/libro-human-cent...,(0),"[Springer 2022-11, , Tapa Blanda, Nuevo]",2023-11-06
269,46598250,Statutory Auditors' Objectivity in Corporate A...,Siddhartha Sankar Saha,549.977,357.485,35,Quedan 50 unidades,https://www.buscalibre.com.co/libro-statutory-...,(0),"[Lap Lambert Academic Publishing, Tapa blanda...",2023-11-06
270,46631218,Regulating Cell to Cell Gap,Siddhartha Mitra,608.492,395.520,35,Quedan 50 unidades,https://www.buscalibre.com.co/libro-regulating...,(0),"[Lap Lambert Acad. Publ., Tapa blanda, Nuevo]",2023-11-06
271,46652869,UHF/VHF Radars studies on Atmospheric Stable a...,Siddarth Shankar Das,538.652,350.124,35,Quedan 50 unidades,https://www.buscalibre.com.co/libro-uhf-vhf-ra...,(0),"[Lap Lambert Academic Publishing, Tapa blanda...",2023-11-06


In [39]:
df.to_csv('/Users/Janus/Documents/GitHub/PortfolioProjects/WebScraping_Project/BuscaLibreScraperDataSet.csv',index=False)
